In [2]:
!sudo pip install bcolz -q

In [3]:
import h5py
import matplotlib.pyplot as plt
import numpy as np
import bcolz
%matplotlib inline

In [4]:
raw = h5py.File('nyu_depth_v2_labeled.mat')
N,channels,width,height = raw['images'].shape

In [5]:
bs = 10
img_mu = np.array([125,107,102])
img_std = 76.4
depth_mu = 2.7
depth_std = 1.4
def prepare_batch(index,bs = 10):
    dat = h5py.File("data.hdf5",'w')
    while True:
        idx = np.sort(np.random.choice(index,bs,replace=False)).tolist()
        img = dat['images'][idx,]

        depth = dat['depths'][idx,]
        depth = np.expand_dims(depth,-1)
        yield img, depth

In [15]:
raw['images'].shape[]

(1449, 3, 640, 480)

In [37]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

images = bcolz.fill( (N,height,width,channels),dtype='float32',rootdir="data/images_bcolz",mode="w")
depths = bcolz.fill( (N,height,width,1),dtype='float32',rootdir="data/depths_bcolz",mode="w")

size = 100

print('transform images..')
for i, img in enumerate(chunks(raw['images'] ,size)):
    img = np.einsum('nkli->nilk', img)
    img = ((img-img_mu)/img_std)
    images[(i*size):min((i+1)*size,N),] = img
    
images.flush()
print('transform depths..')
for i, depth in enumerate(chunks(raw['depths'] ,size)):
    depth = np.einsum('nkl->nlk', depth)
    depth = (depth-depth_mu)/1.5
    depth = np.expand_dims(depth,-1)
    depths[(i*size):min((i+1)*size,N),] = depth
depths.flush()
        
bs = 10
img_mu = np.array([125,107,102])
img_std = 76.4
depth_mu = 2.7
depth_std = 1.4

transform images..
transform depths..
